## LFP Extraction


In [ ]:

import pynwb
import os
import numpy as np
# DataJoint and DataJoint schema
import datajoint as dj
dj.config['filepath_checksum_size_limit'] = 1 * 1024**2

## We also import a bunch of tables so that we can call them easily
from spyglass.common import (
    RawPosition,
    StateScriptFile,
    VideoFile,
    DataAcquisitionDevice,
    CameraDevice,
    Probe,
    DIOEvents,
    ElectrodeGroup,
    Electrode,
    Raw,
    SampleCount,
    FirFilter,
    IntervalList,
    Lab,
    LabMember,
    LabTeam,
    Institution,
    BrainRegion,
    SensorData,
    Session,
    Subject,
    Task,
    TaskEpoch,
    Nwbfile,
    AnalysisNwbfile,
    NwbfileKachery,
    AnalysisNwbfileKachery,
    interval_list_contains,
    interval_list_contains_ind,
    interval_list_excludes,
    interval_list_excludes_ind,
    interval_list_intersect,
    get_electrode_indices,
)

from spyglass.lfp.v1 import (
    LFPElectrodeGroup,
    LFPSelection,
    LFP,
    LFPOutput,
    LFPBandSelection, 
    LFPBand
)

import warnings

warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=ResourceWarning)


#### Next we select the NWB file, which corresponds to the dataset we want to extract LFP from

In [ ]:
nwb_file_names = Nwbfile().fetch("nwb_file_name")
# take the first one for this demonstration
nwb_file_name = nwb_file_names[0]
print(nwb_file_name)

# test: 
nwb_file_name = 'tonks20211103_.nwb'


#### Create the standard LFP Filters. This only needs to be done once.

In [ ]:
FirFilter().create_standard_filters()


#### Now we create an lfp electrode group that defines the set of electrodes we want to filter for lfp data. 
In this case we'll take the three electrode with indexes 0, 4, and 8. 

In [ ]:
LFPOutput().LFP()

In [ ]:
electrode_ids = (Electrode & {"nwb_file_name": nwb_file_name}).fetch("electrode_id")
#electrode_indexes = [0, 4, 8]
#lfp_electrode_ids = electrode_ids[electrode_indexes]
lfp_electrode_ids =  [28, 32, 40]
lfp_electrode_group_name = "test_group"


lfp_eg_key = {"nwb_file_name" : nwb_file_name, "lfp_electrode_group_name" : lfp_electrode_group_name}
#Delete the old test group if it exists (uncomment the line below if so) and then insert the new one
#(LFPElectrodeGroup & lfp_eg_key).delete(force_parts=True)
LFPElectrodeGroup.create_lfp_electrode_group(nwb_file_name=nwb_file_name, group_name=lfp_electrode_group_name, electrode_list=lfp_electrode_ids)


#### We now look at the list of electrodes that are part of this lfp electrode group to verify that we got the right ones

In [ ]:
LFPElectrodeGroup().LFPElectrode() & {"nwb_file_name": nwb_file_name}

#### Next we need to select an interval list and the lfp filter we want to use
You might need to run<br>
(IntervalList & {"nwb_file_name": nwb_file_name}) <br>
    to see the list of intervals and similarly<br>
FIRFilter() <br>
    to see the list of defined filters

In [ ]:
# we choose the first run period and the standard LFP filter for 30KHz data and add a new short interval for this demonstration
orig_interval_list_name = "02_r1"
valid_times = (IntervalList & {"nwb_file_name" : nwb_file_name, "interval_list_name" : orig_interval_list_name}).fetch1("valid_times")
new_valid_times = np.asarray([[valid_times[0,0], valid_times[0,0]+100]])
interval_list_name = "test interval"
IntervalList.insert1({"nwb_file_name":nwb_file_name, "interval_list_name":interval_list_name, "valid_times":new_valid_times}, skip_duplicates=True)

filter_name = "LFP 0-400 Hz"
filter_sampling_rate = 30000


#### Now we create the LFPSelection entry to combine the data, interval list and filter

In [ ]:
lfp_s_key = lfp_eg_key.copy()
lfp_s_key["target_interval_list_name"] = interval_list_name
lfp_s_key["filter_name"] = filter_name
lfp_s_key["filter_sampling_rate"] = filter_sampling_rate
LFPSelection.insert1(lfp_s_key, skip_duplicates=True)

### Populate the LFP table. Note that this takes 2 hours or so on a laptop if you use all electrodes

Note here that populating the LFP table also inserts an LFP entry into LFPOutput, a table that allows us to merge computed and imported lfps

In [ ]:
LFP().populate(lfp_s_key)
LFPOutput()

### Now that we've created the LFP object we can perform a second level of filtering for a band of interest, in this case the theta band
We first need to create the filter

In [ ]:
lfp_sampling_rate = (LFP() & {"nwb_file_name": nwb_file_name}).fetch1(
    "lfp_sampling_rate"
)
filter_name = "Theta 5-11 Hz"
FirFilter().add_filter(
    filter_name,
    lfp_sampling_rate,
    "bandpass",
    [4, 5, 11, 12],
    "theta filter for 1 Khz data",
)


Next we add an entry for the LFP Band and the electrodes we want to filter

In [ ]:


# assume that we've filtered these electrodes; change this if not
lfp_band_electrode_ids =  [28, 32]

# set the interval list name for this band; here we use the same interval as above
interval_list_name = "test interval"

# set the reference to -1 to indicate no reference for all channels
ref_elect = [-1]

# desired sampling rate
lfp_band_sampling_rate = 100

# we also need the uuid for the LFP object
lfp_id = (LFPOutput.LFP & lfp_s_key).fetch1("lfp_id")

In [ ]:
LFPBandSelection().set_lfp_band_electrodes(
    nwb_file_name=nwb_file_name,
    lfp_id=lfp_id,
    electrode_list=lfp_band_electrode_ids,
    filter_name=filter_name,
    interval_list_name=interval_list_name,
    reference_electrode_list=ref_elect,
    lfp_band_sampling_rate=lfp_band_sampling_rate
)
lfp_b_key = (LFPBandSelection & {"lfp_id": lfp_id, "filter_name" : filter_name}).fetch1("KEY")

Check to make sure it worked

In [ ]:
(LFPBandSelection() & {"nwb_file_name": nwb_file_name})


In [ ]:
LFPBand().populate(LFPBandSelection() & {"nwb_file_name": nwb_file_name})
LFPBand()


### Now we can plot the original signal, the LFP filtered trace, and the theta filtered trace together.
Much of the code below could be replaced by a function calls that would return the data from each electrical series

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# get the three electrical series objects and the indeces of the electrodes we band pass filtered
orig_eseries = (Raw() & {"nwb_file_name": nwb_file_name}).fetch_nwb()[0]["raw"]
orig_elect_indeces = get_electrode_indices(orig_eseries, lfp_band_electrode_ids)
orig_timestamps = np.asarray(orig_eseries.timestamps)

lfp_eseries = (LFP() & lfp_s_key).fetch_nwb()[0]["lfp"]
lfp_elect_indeces = get_electrode_indices(lfp_eseries, lfp_band_electrode_ids)
lfp_timestamps = np.asarray(lfp_eseries.timestamps)

lfp_band_eseries = (LFPBand() & lfp_b_key).fetch_nwb()[0][
    "filtered_data"
]
lfp_band_elect_indeces = get_electrode_indices(lfp_band_eseries, lfp_band_electrode_ids)
lfp_band_timestamps = np.asarray(lfp_band_eseries.timestamps)

In [ ]:
# get a list of times for the first run epoch and then select a 2 second interval 100 seconds from the beginning
#run1times = (
#    IntervalList & {"nwb_file_name": nwb_file_name, "interval_list_name": "02_r1"}
#).fetch1("valid_times")
plottimes = [new_valid_times[0][0] + 10, new_valid_times[0][0] + 12]


In [ ]:
# get the time indeces for each dataset
orig_time_ind = np.where(
    np.logical_and(
        orig_timestamps > plottimes[0], orig_timestamps < plottimes[1]
    )
)[0]

lfp_time_ind = np.where(
    np.logical_and(
        lfp_timestamps > plottimes[0], lfp_timestamps < plottimes[1]
    )
)[0]
lfp_band_time_ind = np.where(
    np.logical_and(
        lfp_band_timestamps > plottimes[0],
        lfp_band_timestamps < plottimes[1],
    )
)[0]


In [ ]:
plt.plot(
    orig_eseries.timestamps[orig_time_ind],
    orig_eseries.data[orig_time_ind, orig_elect_indeces[0]],
    "k-",
)
plt.plot(
    lfp_eseries.timestamps[lfp_time_ind],
    lfp_eseries.data[lfp_time_ind, lfp_elect_indeces[0]],
    "b-",
)
plt.plot(
    lfp_band_eseries.timestamps[lfp_band_time_ind],
    lfp_band_eseries.data[lfp_band_time_ind, lfp_band_elect_indeces[0]],
    "r-",
)
plt.xlabel("Time (sec)")
plt.ylabel("Amplitude (AD units)")

plt.show()


#### Now we delete the tutorial entries we added to clean up the database 

In [ ]:
LFPOutput.delete({"lfp_id": lfp_id})
LFPElectrodeGroup.delete(lfp_eg_key)